In [ ]:
import ipyparallel as ipp
c = ipp.Client()
c.ids

In [ ]:
%px %matplotlib inline

In [ ]:
%%px
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

print('Rank {}/{} is alive.'.format(rank, size))

from IPython.display import display, clear_output
import sys

print("Python executable:", sys.executable)
print("Python version:", sys.version)

import math
import time

In [ ]:
%%px

TOTAL_TIME = 0.1
VERBOSE1=True
VERBOSE2=False

Cs=math.sqrt(1/3)
D=1e-3 #m
L=1 #m

D_nd=50 #100

Yn=D_nd #+1
Xn=200 #int(Yn*L/D)

if(VERBOSE1): print("Ny: {0}".format(Yn))
if(VERBOSE1): print("Nx: {0}".format(Xn))

dx=D/D_nd #old->5*10**(-5)
dy = dx
if(VERBOSE1): print("dx: {0}".format(dx))
#relaxation time tau, should be > 0,5
tau=0.6

dP=1e-2 #Pa
rho_0=1e3 #kg/m^3
p_in=1/3
p_out=p_in-dP
roh_in=p_in/Cs**2
roh_out=p_out/Cs**2
dRho=dP/Cs**2


if(VERBOSE1): 
    print("p_in: {0}".format(p_in))
    print("p_out: {0}".format(p_out))
    print("roh_in: {0}".format(roh_in))
    print("roh_out: {0}".format(roh_out))

nu=2.9e-6 #m^2/s 
dt=Cs**2*(tau-0.5)*(dx**2/nu)

nu_ = Cs**2*(tau-0.5)*(dx**2) * dx
dt_=Cs**2*(tau-0.5)*(dx**2/nu_)

Cs_ = math.sqrt(1/3*(dx**2/dt**2))
if(VERBOSE1): print("dt: {0}".format(dt))

#Poiseuille centerline (max) velocity
U=1/8*(rho_0/nu)*(dP/L)*(D**2)
#U=1.25

Re=D*U/nu
Ma=U/Cs 
Kn=U*D/nu
if(VERBOSE1): 
    print("U: {0}".format(U))
    print("Re: {0}".format(Re))
    print("Ma: {0}".format(Ma))
    print("Kn: {0}".format(Kn))

#we need Cl, Croh, Ct

# 1. Conversion factor Cl for length
Cl=dx #freely chosen
dx_nd=dx/Cl
if(VERBOSE1): 
    print("Cl: {0}".format(Cl))
    print("dx_nd: {0}".format(dx_nd))

#2. Conversion factor Croh for density
Croh=rho_0
roh_nd = rho_0/Croh
if(VERBOSE1): 
    print("Croh: {0}".format(Croh))
    print("roh_nd: {0}".format(roh_nd))

#3. Conversion factor Ct for time
Ct=dt
dt_nd = dt/Ct
if(VERBOSE1): 
    print("Ct: {0}".format(Ct))
    print("dt_nd: {0}".format(dt_nd))

#4. Conversion factor Cu for velocity
Cu=Cl/Ct
U_nd = U/Cu #-> limit U_nd=0.1
U_nd=0.1

if(VERBOSE1): 
    print("Cu: {0}".format(Cu))
    print("U_nd: {0}".format(U_nd))

#5. Conversion factor CF for Force
CF=Croh*Cl**4*Ct**(-2)
if(VERBOSE1): print("CF: {0}".format(CF))

#6. Conversion factor Cf for frequency
Cf=1/Ct
if(VERBOSE1): print("Cf: {0}".format(Cf))

#change nu_nd in order to achieve U_nd=0,1
nu_nd=((D_nd*U_nd)/(D*U))*nu
if(VERBOSE1): print("nu_nd: {0}".format(nu_nd))

tau_nd=(nu_nd/Cs**2)+1./2
if(VERBOSE1): print("tau_nd: {0}".format(tau_nd))
omega = dt/tau
if(VERBOSE1): print("omega: {0}".format(omega))
omega_nd = dt_nd/tau_nd
if(VERBOSE1): print("omega_nd: {0}".format(omega_nd))

In [ ]:
%%px

iteration = 0

columns_to_select = [1, 2, 3, 10, 20, 50, Xn-1]
_roh_at_points_top = []
_roh_at_points_mid = []
_roh_at_points_bottom = []

#discrete velocity channels for D2Q9
discrete_velocities = np.array([[0, 0],     # i=0
                      [1, 0],               # i=1
                      [0, 1],               # i=2
                      [-1, 0],              # i=3
                      [0, -1],              # i=4
                      [1, 1],               # i=5
                      [-1, 1],              # i=6
                      [-1, -1],             # i=7
                      [1, -1]])             # i=8

if(VERBOSE1): print("Discrete velocities: {0}".format(discrete_velocities))

#weights
weights = np.array([4/9,1/9,1/9,1/9,1/9,1/36,1/36,1/36,1/36])
if(VERBOSE1): print("Weights: {0}".format(weights))

In [ ]:
%%px

#equilibrium distribution function feq(0->8)
def f_eq_2D(_rho, _u_ckl):
    global discrete_velocities, weights
    
    f_eq = (
        weights.reshape((9, 1)) * np.ones(_rho.shape) *
        _rho.reshape(1, *(_rho.shape)) *
        (
            np.ones(np.dot(discrete_velocities, _u_ckl) .shape) +
            np.dot(discrete_velocities, _u_ckl) / Cs**2 +
            (1/2.) * np.dot(discrete_velocities, _u_ckl) ** 2 / Cs**4 -
            (1/2.) * np.einsum('ki,ki->i', _u_ckl, _u_ckl).reshape(1, *(np.einsum('ki,ki->i', _u_ckl, _u_ckl).shape)) / Cs**2
        )
    )

    return f_eq


def f_eq_3D(_rho, _u_ckl):
    global discrete_velocities, weights

    _u_ckl_dot = np.einsum('hk,kij->hij', discrete_velocities, _u_ckl) #(9,2) * (2,101,101002)
    _u_ckl_product = np.einsum('kij,kij->ij', _u_ckl, _u_ckl)
    _u_ckl_product_reshaped = _u_ckl_product.reshape(1, *(_u_ckl_product.shape))
    _ones = np.ones(_u_ckl_dot.shape)

    _rho_reshaped = _rho.reshape(1, *(_rho.shape))
    weights_reshaped = weights.reshape((9, 1, 1)) * np.ones(_rho.shape)
    factors = weights_reshaped * _rho_reshaped

    #Part_2_BTE_to_LBM, BTE3
    #feq = factors * (
    #    _ones + 3. * _u_ckl_dot / Cs**2 + (9. / 2.) * _u_ckl_dot**2 / Cs**4 - (3. / 2.) * _u_ckl_product_reshaped / Cs**2
    #)

    #BGK formula, Kruger pp67
    f_eq = factors * (
        #_ones + _u_ckl_dot / Cs**2 + (1/2.) * _u_ckl_dot**2 / Cs**4 - (1/2.) * _u_ckl_product_reshaped / Cs**2
        _ones + _u_ckl_dot / Cs**2 + (1/2.) * _u_ckl_dot**2 / Cs**4 - (1/2.) * _u_ckl_product / Cs**2
    )


    #c_u = np.einsum('vi, xyi -> xyv', c, u)
    #c_u_2 = c_u ** 2 # np.einsum('xyv, xyv -> xyv', c_u, c_u)
    #u_2 = np.einsum('jki, jki -> jk', u, u)

    #f_eq = np.zeros((nx + 2, ny + 2, 9))
    #for i in range(9):
    #    f_eq[:,:,i] = w_i[i] * rho * (1 + 3 * c_u[:,:,i] + 9 / 2 * c_u_2[:,:,i] - 3 / 2 * u_2)
    
    return f_eq


def communicate(_ltc):
    VERBOSE2 = False

    #Lösung nach Herrn Greiner
    if False:
        #Send to the right, receive from the left
        _sendbufR = np.ascontiguousarray(_ltc[:, -2, :].copy(), dtype=np.float32)
        _recvbufR = np.ascontiguousarray(_ltc[:, 0, :].copy(), dtype=np.float32)
        cartcomm.Sendrecv(_sendbufR, destR, recvbuf=_recvbufR, source=sourceR)
        _ltc[:,0,:] = _recvbufR
        if VERBOSE2:
            print(f"----Send to the right, receive from the left----------------------")                    
            print(f"Rank {rank}: destR={destR}")
            print(f"Rank {rank}: sourceR={sourceR}")
            print(f"Rank {rank}: _sendbufR={_sendbufR[:,0,0]}")
            print(f"Rank {rank}: _recvbufR={_recvbufR[:,0,0]}")        
    
        #Send to the left, receive from the right
        _sendbufL = np.ascontiguousarray(_ltc[:, 1, :].copy(), dtype=np.float32)
        _recvbufL = np.ascontiguousarray(_ltc[:, -1, :].copy(), dtype=np.float32)
        cartcomm.Sendrecv(_sendbufL, destL, recvbuf=_recvbufL, source=sourceL)
        _ltc[:, -1, :] = _recvbufL.reshape(_recvbufL.shape[0], _recvbufL.shape[-1])  
        if VERBOSE2:
            print(f"---Send to the left, receive from the right--------------------------")                    
            print(f"Rank {rank}: destL={destL}")
            print(f"Rank {rank}: sourceL={sourceL}")
            print(f"Rank {rank}: _sendbufL={_sendbufL[:,0,0]}")
            print(f"Rank {rank}: _recvbufL={_recvbufL[:,0,0]}")           

    #Lösung nach Herrn Pastewka
    if True:
        #Send to the right, receive from the left
        _sendbufR = np.ascontiguousarray(_ltc[:, -2:-1, :].copy(), dtype=np.float32)
        _recvbufR = np.ascontiguousarray(_ltc[:, 0:1, :].copy(), dtype=np.float32)
        cartcomm.Sendrecv(_sendbufR, destR, recvbuf=_recvbufR, source=sourceR)
        _ltc[:,0:1,:] = _recvbufR
        if VERBOSE2:
            print(f"----Send to the right, receive from the left----------------------")                    
            print(f"Rank {rank}: destR={destR}")
            print(f"Rank {rank}: sourceR={sourceR}")
            print(f"Rank {rank}: _sendbufR={_sendbufR}")
            print(f"Rank {rank}: _recvbufR={_recvbufR}")
    
        #Send to the left, receive from the right
        _sendbufL = np.ascontiguousarray(_ltc[:, 1:2, :].copy(), dtype=np.float32)
        _recvbufL = np.ascontiguousarray(_ltc[:, -1:, :].copy(), dtype=np.float32)
        cartcomm.Sendrecv(_sendbufL, destL, recvbuf=_recvbufL, source=sourceL)
        _ltc[:, -1, :] = _recvbufL.reshape(_recvbufL.shape[0], _recvbufL.shape[-1])           
        if VERBOSE2:
            print(f"---Send to the left, receive from the right--------------------------")                    
            print(f"Rank {rank}: destL={destL}")
            print(f"Rank {rank}: sourceL={sourceL}")
            print(f"Rank {rank}: _sendbufL={_sendbufL}")
            print(f"Rank {rank}: _recvbufL={_recvbufL}")            

    return _ltc


#roll the lattice based on the discrete velocities
def streamLattice0(_ltc):
    global discrete_velocities, weights
    
    shifted_lattice = np.stack([
        np.roll(np.roll(_ltc[d, :, :], shift=dx, axis=0), shift=dy, axis=1)
        for d, (dx, dy) in enumerate(discrete_velocities)
    ], axis=0)

    return shifted_lattice


def streamLattice1(f):
    ''' Performs the streaming step of the boltzmann transport equation

    Arguements
    -----------
    f: np.array (9, nx, ny)
        array containing the probability density of each grid point

    Returns
    ----------
    f: np.array (9, nx, ny)
        array containing the streamed probability density of each grid point  
    '''
    global discrete_velocities
    for i in range(1, 9):
        f[i, :, :] = np.roll(f[i, :, :], discrete_velocities[i], axis=(0, 1))
    return f       


#update the first 2 moments (density, current density, _u_ckl)
def updateMoments(_ltc):
    global discrete_velocities, weights

    #print(f"Rank {rank}: _ltc.shape={_ltc.shape}, size={_ltc.nbytes} bytes")
   
    # density:
    _roh = np.sum(_ltc, axis=0)

    # current density:
    _current_density =  np.einsum('ki,ijl->kjl', discrete_velocities.T, _ltc)

    # average velocity:
    _u_ckl = _current_density / _roh

    return _roh, _u_ckl


def bounceBackTopBottom2(f, nx, ny):
    '''Performs the bounce back step
    
    Arguements
    -----------
    f: np.array (nx, ny, 9)
        probability density function
    nx: int
        number of grid points in x direction
    ny: int
        number of grid points in y direction
    
    Returns
    ---------
    f: np.array (nx, ny, 9)
        probability density function after the bounce back step
    '''
     
    # rigid lower wall 
    f[2, 1 : nx + 1, 1] = f[4, 1 : nx + 1, 0]
    f[5, 1 : nx + 1, 1] = np.roll(f[7, 1 : nx + 1, 0], 1)
    f[6, 1 : nx + 1, 1] = np.roll(f[8, 1 : nx + 1, 0], -1)
    
    # rigid upper wall
    f[4, 1 : nx + 1, ny] = f[2, 1 : nx + 1, ny + 1]
    f[7, 1 : nx + 1, ny] = np.roll(f[5, 1 : nx + 1, ny + 1], -1)
    f[8, 1 : nx + 1, ny] = np.roll(f[6, 1 : nx + 1, ny + 1], 1)

    return f    


#calulate boundary nodes X(0) and X(N+1) for periodic BC with presssure difference
def calcPeriodicBCN(_roh_1, _u_c1l, _roh_out, _ltc_0): 

    f_eq_out = f_eq_2D(_roh_out, _u_c1l)
    fi_x0_prestream = _ltc_0
    fi_eq_1 = f_eq_2D(_roh_1, _u_c1l)
    fi_xNplus1 = f_eq_out + (fi_x0_prestream - fi_eq_1)     

    return fi_xNplus1 #fi_x0, fi_xNplus1


#calulate boundary nodes X(0) and X(N+1) for periodic BC with presssure difference
def calcPeriodicBC0(_roh_N, _u_cNl, _roh_in, _ltc_N): 

    f_eq_in = f_eq_2D(_roh_in, _u_cNl)
    fi_xN_prestream = _ltc_N
    fi_eq_N = f_eq_2D(_roh_N, _u_cNl)
    fi_x0 = f_eq_in + (fi_xN_prestream - fi_eq_N)

    return fi_x0 #fi_x0, fi_xNplus1 #fi_x


#2D Poiseuille inlet velocity u(y) for comparison with numerical result
def Poiseuille2DUy(y):
    u_poiseuille = np.zeros((2), dtype=float)
    u_y = U * (1 - ((y - D/2)/(D/2))**2)
    u_poiseuille[0] = u_y
    u_poiseuille[1] = 0

    return u_poiseuille   


def Poiseuille2DUy2(y, U, D):
    """
    Returns the velocity profile for Poiseuille flow in a pipe.
    :param y: Array of radial distances (y values).
    :param U: Maximum velocity at the center of the pipe.
    :param D: Diameter of the pipe.
    :return: Velocity profile at each y position.
    """
    R = D / 2  # Pipe radius
    u_poiseuille = U * (1 - (y / R) ** 2)
    return u_poiseuille  


def get_adjusted_velocity_y_values4Poiseuille2D(num_nodes1, D):
    # Define the spacing factor for this array as well
    delta = D / (num_nodes1 - 1)
    R = D / 2.0
    
    # Initialize the velocity_y_values array with proper non-uniform spacing
    velocity_y_values4Poiseuille2D = np.zeros(num_nodes1)
    
    # Middle points
    velocity_y_values4Poiseuille2D = np.linspace(-R, R, num_nodes1)
    
    return velocity_y_values4Poiseuille2D


def plot_multiple_frames_amplitude(list, list_frames, fig_size, axis, xlabel, ylabel, nx, ny):
    '''Plots the amplitude of multiple frames
    
    Arguements
    -----------
    list: list of arrays
        list containing the density or velocity at each simulation step
    list_frames: list of ints
        frames which are to be plotted
    fig_size: tuple of ints
        size of the figure
    axis: np.array
        x axis for plotting
    nx, ny: int
        number of grid points in x and y direction
    xlabel, ylabel: str
        labels for plotting

    Returns 
    -------
    None
    '''
    plt.figure(figsize = fig_size)
    # axis = np.hstack((axis, np.array([nx])))
    for n in range(len(list_frames)):
        u_or_rho = list[list_frames[n]][0, nx // 2, 1 : ny + 1] # u_or_rho = np.hstack((list[list_frames[n]][:,L // 4], np.array(list[list_frames[n]][0][L//4])))
        plt.plot(axis, u_or_rho, label = "t = " + str(list_frames[n]))
    plt.legend(ncol = len(list_frames) // 4, loc = 'upper right')
    plt.grid()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)  


def density_plot(den_eq, nx, ny):
    # Plot the density profile
    fig, ax = plt.subplots(figsize = (17, 5))
    #plt.figure()
    ax.plot(den_eq[1:nx, ny // 2])
    #plt.plot(np.arange(1, nx + 1), den_eq[1 : nx + 1, ny // 2])
    #plt.grid()
    #plt.xlabel('x Achse')
    #plt.ylabel('Dichte')


def collect_pdf(__pdf, _pdf_full_range):
   
    _sendBuf_pdf = np.ascontiguousarray(__pdf[:, 1:-1, :], dtype=np.float32)

    # Allocate _pdf_full_range correctly
    if rank == 0:    
        assert _pdf_full_range.flags['C_CONTIGUOUS']  
        
    cartcomm.Gather(_sendBuf_pdf, _pdf_full_range, root=0)
    return _pdf_full_range

In [ ]:
%%px

VERBOSE = False

#Xn=200
nlocal = Xn//size
if rank == size-1:
    nlocal = Xn - nlocal*(size-1)
nx1 = nlocal*rank
nx2 = nlocal*(rank+1)
if rank==size-1:
    nx2 = Xn
x = np.arange(nx1-1,nx2+1)
if VERBOSE:
    print("x: ")
    print(x)

#preliminary
#lattice for phase space; Nx+3 is due to periodic boundary conditions
#Nx is the number of divisions in the x-direction, thus there are Nx+3 points when including the extra nodes 0 and N+1 in x-direction
#lattice columns start with 0 and end with Nx+2, X(0) = X(0) and X(N+1) = X(Nx+2)

#initialise
#average velocity, cartesion x,y-directions, k is y-position, l is x-position
INIT_ROH = 1 #0.001

roh_in_k = np.full((Yn+2), roh_in, dtype=np.float32)
roh_out_k = np.full((Yn+2), roh_out, dtype=np.float32)

u_ckl = np.zeros((2, len(x), Yn+2), dtype=np.float32)    
roh = np.full((len(x), Yn+2), INIT_ROH, dtype=np.float32)

if rank == 0:
    roh[0,:] = roh_in_k    
elif rank == (size-1):    
    roh[-1,:] = roh_out_k    

pdf = f_eq_3D(roh, u_ckl)
print(f"Rank {rank}: pdf.shape={pdf.shape}, pdf.size={pdf.nbytes} bytes")   

# Simulation parameters
num_iterations = int(TOTAL_TIME / dt) + 1
num_nodes1 = Yn #int(D/dy) + 1  # Number of nodes in inlet/outlet
R = D / 2  # Radius of the pipe
num_nodes2 = columns_to_select

scaled_x = None
if rank == 0:
    num_x_values = Xn+2 #roh.shape[0]
    x_all = np.linspace(0, L, num_x_values)  # Example of all possible x-values
    scaled_x = x_all[columns_to_select]  # Get true x-values for selected columns

if rank == 0:
    velocity_y_values = range(1, Yn+1)    
    
if VERBOSE:
    print("num_nodes1:")
    print(num_nodes1)
    print("velocity_y_values:")
    print(velocity_y_values)
if rank == 0:
    velocity_y_values4Poiseuille2D = get_adjusted_velocity_y_values4Poiseuille2D(num_nodes1, D)
    if VERBOSE:
        print("velocity_y_values4Poiseuille2D:")
        print(velocity_y_values4Poiseuille2D)


#cartesian comunicator
cartcomm = comm.Create_cart(dims=[size],periods=[True],reorder=False)
rcoords=cartcomm.Get_coords(rank)
sourceR,destR=cartcomm.Shift(0,1)
print(f"sourceR, destR: {sourceR},{destR}")
sourceL,destL=cartcomm.Shift(0,-1)
print(f"sourceL, destL: {sourceL},{destL}")


x_labels = [1, 2, 3, 10, 20, 50, Xn]
min_value = 0
density_max = 1.5


idt = None
if rank == 0:
    idt = int(TOTAL_TIME / dt) + 1  # Compute number of steps
else:
    idt = None
# Broadcast idt from rank 0 to all other ranks
idt = comm.bcast(idt, root=0)
# Generate the time range (each process gets the same range)
t_range = np.linspace(0, TOTAL_TIME, idt)

_u_cNl = None
_roh_kN = None
fi_xN_prestream = None
_u_c0l = None
_roh_k0 = None
fi_x0_prestream = None


#arrays for comm.Collect
_pdf_full_range = None
_u_ckl_full_range = None
if rank == 0:
    _pdf_full_range = np.zeros((9, Xn+2, Yn+2), dtype=np.float32)
    _pdf_full_range = np.ascontiguousarray(_pdf_full_range, dtype=np.float32)
    
    _u_ckl_full_range = np.zeros((2, Xn+2, Yn+2), dtype=np.float32)
    _u_ckl_full_range = np.ascontiguousarray(_u_ckl_full_range, dtype=np.float32)
            

VERBOSE1 = False
TOTAL_ITERATION = 12001
start = time.perf_counter()

fi_x0 = None
fi_xNplus1 = None
#list_streaming = None
list_streaming = [u_ckl]
w = 1

comm.Barrier()         

for t in t_range:

    iteration = comm.bcast(iteration, root=0)    

    
    #1. moment update
    if iteration > 0:
        roh, u_ckl = updateMoments(pdf) #[:, 1 : nlocal + 1, 1 : Yn + 1]


    # Get the maximum density and its location
    max_density = np.max(roh)
    max_location = np.unravel_index(np.argmax(roh), roh.shape)
    if np.any(roh > 1):
        if VERBOSE1 and rank == 0:
            print(f"Instability detected at iteration {iteration + 1}")
    if VERBOSE1 and rank == 0:
        print(f"Maximum density: {max_density} at location {max_location}")

    
    #2. compute equilibrium
    f_eq = f_eq_3D(roh, u_ckl)

    #3. collision term
    #pdf[:, 1 : nlocal + 1, 1 : Yn + 1] = pdf[:, 1 : nlocal + 1, 1 : Yn + 1] - pdf[:, 1 : nlocal + 1, 1 : Yn + 1]*omega_nd + omega_nd*f_eq[:, 1 : nlocal + 1, 1 : Yn + 1]
    pdf = pdf * (1 - w) + w * f_eq 

    
    #4.1a Periodic Boundary conditions inlet/outlet with pressure difference
    #update extra node layers 0 and N+1 -> A) & B) acc. to Script: Boundary Conditions for the Lattice Boltzmann Method
    #u_ckl profiles at outlet and inlet
       
    #assign inlet and outlet boundary values -> A)    
    #update extra node layers 0 and N+1 -> A) & B) acc. to Script: Boundary Conditions for the Lattice Boltzmann Method

    #assign inlet and outlet boundary values -> A)    
    fi_x0 = None
    fi_xNplus1 = None

    if rank==0:
        _u_cNl = u_ckl[:, 1, :].copy()
        _roh_kN = roh[1, :].copy()
        fi_xN_prestream = pdf[:,-2,:].copy()

    if rank==(size-1):
        _u_c0l = u_ckl[:, -2, :].copy()
        _roh_k0  = roh[-2, :].copy()
        fi_x0_prestream = pdf[:,1,:].copy()

    if rank == 0:   
        #fi_xNplus1 = calcPeriodicBC(_roh_kN, _u_cNl, roh_in_k, _roh_k1, _u_c1l, roh_out_k, _ltc_pre)  
        fi_x0 = calcPeriodicBC0(_roh_kN, _u_cNl, roh_in_k, fi_xN_prestream)

    #assign inlet and outlet boundary values -> A)    
    if rank == (size-1):
        #fi_x0 = calcPeriodicBC(_roh_kN, _u_cNl, roh_in_k, _roh_k1, _u_c1l, roh_out_k, _ltc_pre)
        fi_xNplus1 = calcPeriodicBCN(_roh_k0, _u_c0l, roh_out_k, fi_x0_prestream)
    
    
    #5. stream lattice 
    #store pre-stream boundary top and bottom values
    pdf = streamLattice0(pdf)
    
    #0. communicate between sub-lattices
    pdf = communicate(pdf)   

    #4.2 Bounce-Back Top and Bottom
    #pdf = bounceBackTopBottom1(pdf, _ltc_pre, index_mapping_top, index_mapping_bottom)
    pdf = bounceBackTopBottom2(pdf, nlocal, Yn) 
    
    
    #4.1b. assign inlet boundary values -> B)
    if rank == 0 and fi_x0 is not None:
        pdf[:, 0, :] = fi_x0

    elif rank == (size-1) and fi_xNplus1 is not None:   
        pdf[:, -1, :] = fi_xNplus1

   
    # Update plots
    list_streaming.append(u_ckl)
    last_den = roh


    if rank == 0:
        iteration += 1
    #if VERBOSE1 and rank == 0:
    #    if (iteration % 100) == 0:
    #        print(f"Simulation Execution -> TOTAL_ITERATION: {TOTAL_ITERATION}; iteration: {iteration}; {((iteration/TOTAL_ITERATION)*100.0):.1f} %")

end = time.perf_counter()
print(f"Execution time: {end - start:.6f} seconds")

plot_multiple_frames_amplitude(list_streaming, [0, 10, 50, 100, 200, 500, 1000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000], (17, 10), np.arange(1, Yn + 1), "y Achse", "Geschwindigkeit u$_x$", nlocal, Yn)
density_plot(last_den, nlocal, Yn)
y = np.arange(0.5, Yn + 1, 1)
rho_in, rho_out, cs_2 = last_den[1, Yn // 2], last_den[nlocal, Yn // 2], 1 / 3
dp = (rho_in - rho_out) * cs_2
print(f"rho_in: {rho_in}")
print(f"rho_out: {rho_out}")
print(f"dp: {dp}")